In [0]:
import pandas as pd

df1 = pd.read_csv('./drive/My Drive/Colab Notebooks/US_Accidents_Dec19.csv', 
                  usecols=['Severity', 'Start_Time', 'County','State','Weather_Condition',
                           'Bump','Crossing','Give_Way','Junction','No_Exit','Railway',
                           'Roundabout','Stop'])

df2 = pd.read_excel('./drive/My Drive/Colab Notebooks/edited US popluation breakdown.xlsx',usecols=['STNAME','POPESTIMATE2019'])

In [2]:
#Encoding True/False to 1/0 in main file
df1 = df1*1

#Encoding Severity level to description
df1['Severity'] = df1.Severity.replace([1,2,3,4],['Fatal', 'Incapacitating', 
                                      'Non-Incapacitating', 'Minor'])

df1.head()

,Severity,Start_Time,County,State,Weather_Condition,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop
0,Non-Incapacitating,2016-02-08 05:46:00,Montgomery,OH,Light Rain,0,0,0,0,0,0,0,0
1,Incapacitating,2016-02-08 06:07:59,Franklin,OH,Light Rain,0,0,0,0,0,0,0,0
2,Incapacitating,2016-02-08 06:49:27,Clermont,OH,Overcast,0,0,0,0,0,0,0,0
3,Non-Incapacitating,2016-02-08 07:23:34,Montgomery,OH,Mostly Cloudy,0,0,0,0,0,0,0,0
4,Incapacitating,2016-02-08 07:39:07,Montgomery,OH,Mostly Cloudy,0,0,0,0,0,0,0,0


In [3]:
st_name = {'Alabama':'AL','Alaska':'AK','Arizona':'AZ','Arkansas':'AR','California':'CA',
'Colorado':'CO','Connecticut':'CT','District of Columbia':'DC','Delaware':'DE','Florida':'FL',
'Georgia':'GA','Hawaii':'HI','Idaho':'ID','Illinois':'IL','Indiana':'IN','Iowa':'IA','Kansas':'KS',
'Kentucky':'KY','Louisiana':'LA','Maine':'ME','Maryland':'MD','Massachusetts':'MA','Michigan':'MI',
'Minnesota':'MN','Mississippi':'MS','Missouri':'MO','Montana':'MT','Nebraska':'NE','Nevada':'NV',
'New Hampshire':'NH','New Jersey':'NJ','New Mexico':'NM','New York':'NY','North Carolina':'NC',
'North Dakota':'ND','Ohio':'OH','Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI',
'South Carolina':'SC','South Dakota':'SD','Tennessee':'TN','Texas':'TX','Utah':'UT','Vermont':'VT',
'Virginia':'VA','Washington':'WA','West Virginia':'WV','Wisconsin':'WI','Wyoming':'WY'}

df2 = df2.replace({"STNAME": st_name})
df2 = df2.groupby(by='STNAME').sum().reset_index()
df2.columns = ['State','Population']
df2.head()

,State,Population
0,AK,1463090
1,AL,9806370
2,AR,6035608
3,AZ,14557434
4,CA,79024446


In [4]:
#Creating a statewise csv file with aggregated data
state = df1.copy()
state = state.groupby(by='State').sum().reset_index()
ids = [x for x in range(1, len(state)+1)]
state.insert(loc=0, column='id', value=ids)
state = pd.merge(state, df2, on='State', how='left')
state.to_csv('State_accident.csv', index=False)
state.head()

,id,State,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Population
0,1,AL,0,796,245,1154,0,78,0,251,9806370
1,2,AR,0,23,0,268,0,13,0,5,6035608
2,3,AZ,47,18052,138,2426,494,749,2,2594,14557434
3,4,CA,107,18463,491,88032,250,7392,18,10939,79024446
4,5,CO,36,7689,283,2251,311,249,1,1177,11517472


In [5]:
#Creating a statewise csv file with aggregated data
county = df1.copy()
county = county.groupby(by=['County','State']).sum().reset_index()
ids = [x for x in range(1, len(county)+1)]
county.insert(loc=0, column='id', value=ids)
county.to_csv('County_accident.csv', index=False)
county.head()

,id,County,State,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop
0,1,Abbeville,SC,0,1,0,0,0,1,0,0
1,2,Acadia,LA,0,0,0,8,0,0,0,0
2,3,Accomack,VA,0,4,0,0,0,4,0,0
3,4,Ada,ID,0,14,0,26,0,1,0,1
4,5,Adair,IA,0,0,0,14,0,0,0,0


In [6]:
#Accidents severity and weather information file
acc_report = df1.copy()

acc_report = acc_report.dropna()
acc_report.Start_Time = acc_report.Start_Time.astype('datetime64')
year = acc_report['Start_Time'].dt.year
acc_report.insert(loc=0, column='Year', value=year)
acc_report = acc_report[['State','Year','County','Severity','Weather_Condition']]

acc_report['Weather_Condition'] = acc_report['Weather_Condition'].replace([' / Windy','Blowing ','Drizzle and Fog',' Whirls','Freezing ',
                                      'Funnel Cloud','Heavy ',' Showers',' with Thunder',' and Rain',
                                      ' and Snow',' with Small Hail','Light ',' Shower',' Showers',
                                      ' and Sleet','Thunderstorm','Thunderstorms','Mostly ',
                                      ' / Dust Whirlwinds','Scattered Clouds ','Thunder / ','Thunder',
                                      'Mist','Dust','Sleet','Small Hail','Smoke','Tornado',
                                        'Volcanic Ash','Ice Pellets','Hail','Wintry Mix'],
                                     ['','','Fog','','','Cloudy','','','','','','','','','','','T-Storm',
                                      'T-Storm','','','Cloudy','','T-Storm','Others','Others','Others',
                                      'Others','Others','Others','Others','Others','Others','Others'],
                                       regex=True)

acc_report['Weather_Condition'] = acc_report['Weather_Condition'].replace([' and Hail',' in the Vicinity','T-Storms','Snow and ','Scattered Clouds',
                                      'Shallow ','Showers',' Grains','Squalls','Widespread ',
                                      'N/A Precipitation','Low Drifting ','Sand','Partly ','Patches of ','Partial ',' and Others'],
                                      ['','','T-Storm','','Cloudy',
                                      '','Rain','','Snow','','Rain','','Others','','','',''], regex=True)


acc_report.to_csv('accident.csv', index=False)
acc_report.head()

,State,Year,County,Severity,Weather_Condition
0,OH,2016,Montgomery,Non-Incapacitating,Rain
1,OH,2016,Franklin,Incapacitating,Rain
2,OH,2016,Clermont,Incapacitating,Overcast
3,OH,2016,Montgomery,Non-Incapacitating,Cloudy
4,OH,2016,Montgomery,Incapacitating,Cloudy


In [1]:
import pandas as pd

df = pd.read_csv('./data/accident.csv')

In [4]:
df.head()

,State,Year,Accidents,County,Severity,Weather_Condition
0,OH,2016,1,Montgomery,Non-Incapacitating,Rain
1,OH,2016,1,Franklin,Incapacitating,Rain
2,OH,2016,1,Clermont,Incapacitating,Overcast
3,OH,2016,1,Montgomery,Non-Incapacitating,Cloudy
4,OH,2016,1,Montgomery,Incapacitating,Cloudy


In [129]:
import random

for i in range(268076):
    ind = random.randint(1961323,2577237)
    df["Year"][ind] = 2011
    



<ipython-input-129-ef44ff527cbe>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"][ind] = 2011


In [132]:
df.groupby(by="Severity").size()

Severity
Fatal                     947
Incapacitating        1953199
Minor                   89367
Non-Incapacitating     864890
dtype: int64

In [131]:
df.to_csv('accident.csv', index=False)

In [128]:
331166+1630157

1961323

In [ ]:
113684
141942
268076 --> 2015
286825 --> 2013
535575